# fcc and bcc Crystals

Structure:
1. create fcc unit cell
2. create bcc unit cell
3. create supercell
4. add a vacancy to the supercell
5. sanity check neighbour distances

TODOs:
* neatify distance tests of structures
* sort out `box`, `box_size`, `box_vectors`
* add documentation

In [ ]:
import Pkg

In [ ]:
Pkg.activate(".")

In [ ]:
Pkg.test("Crystal")

In [ ]:
Pkg.status()

In [ ]:
using Crystal

In [ ]:
using Molly
using DataFrames
using LaTeXStrings
using Plots

## Creating Crystals

In [ ]:
masses = Dict("V" => 50.9415, "Nb" => 92.9064, "Ta" => 180.9479,
              "Cr" => 51.996, "Mo" => 95.94, "W" => 183.85,
              "Fe" => 55.847)

### An atom

In [ ]:
element = "Fe"
Atom(name=element, mass=masses[element])

## Synthesizing crystal unit cells

Crystals are something fascinating. Defect free crystals are highly symmetric and can be reduced to so-called "unit cells", a cell which can be used by copying and shifting it to construct the entire crystal. So to sound impressive the crystal of multiple unit cells is called a supercell ¯\\\_(ツ)\_/¯.

So in a first step we'll define how to create two common types of unit cells and then go on to synthesize a supercell.

In [ ]:
# Å
bcc_lattice_constants = Dict(
    "V" => 3.0399, "Nb" => 3.3008, 
    "Ta" => 3.3058, "Cr" => 2.8845, "Mo" => 3.1472, 
    "W" => 3.1652, "Fe" => 2.8665
)

### face centered cubic crystal

In [ ]:
element = "Fe"
a = bcc_lattice_constants[element]
atoms, coords, box, box_size, box_vectors = Crystal.make_fcc_unitcell(element, a=a)

In [ ]:
Crystal.plot_crystal(atoms, coords, default_color="red", default_size=50)

### body centered cubic crystal

In [ ]:
element = "Fe"
a = bcc_lattice_constants[element]
atoms, coords, box, box_size, box_vectors = Crystal.make_bcc_unitcell(element, a=a)

In [ ]:
Crystal.plot_crystal(atoms, coords, default_size=50)

In [ ]:
element = "Fe"
a = bcc_lattice_constants[element]
atoms, coords, box, box_size, box_vectors = Crystal.make_fcc_unitcell(element, a=a)

## Inserting a vacancy / removing an atom 

In [ ]:
atoms_vac, coords_vac = Crystal.add_vacancies(atoms, coords, random=true)
@assert length(atoms_vac) == length(atoms) - 1
@assert length(atoms_vac) == length(coords_vac) 

## Generating a supercell from a unit cell

In [ ]:
sc_atoms, sc_coords, sc_box, sc_box_size = Crystal.make_supercell(atoms, coords, box, box_size, nx=3, ny=3,
        nz=3);

In [ ]:
@assert length(sc_atoms) == length(sc_coords)

In [ ]:
Crystal.plot_crystal(sc_atoms, sc_coords, default_size=10)

In [ ]:
atoms_vac, coords_vac = Crystal.add_vacancies(sc_atoms, sc_coords, ixs=[1]);

In [ ]:
Crystal.plot_crystal(atoms_vac, coords_vac, default_size=10)

In [ ]:
n_atoms = length(sc_atoms)

Looks okay so far, let's move on.

## Sanity checking distances

Let's define some minimal objects (`MinimalSimulationConfig`) so we can perform neighbour search in a similar way as is actually done for simulations, but without needing to define interactions.

In [ ]:
element = "Fe"
a = 1 #bcc_lattice_constants[element]
atoms, coords, box, box_size, box_vectors = Crystal.make_fcc_unitcell(element, a=a)
sc_atoms, sc_coords, sc_box, sc_box_size = Crystal.make_supercell(atoms, coords, box, box_size, nx=3, ny=3,nz=3)
n_atoms = length(sc_atoms);

In [ ]:
dist_cutoff = 2
rs_df, rs = Crystal.get_distance_df(sc_atoms, sc_box_size[1,1], sc_coords, dist_cutoff=dist_cutoff)

### fcc

In [ ]:
@assert rs_df.distances[1] ≈ sqrt(1^2+1^2)/2
@assert rs_df.distances[2] ≈ 1
@assert rs_df.distances[3] ≈ sqrt(1^2+(sqrt(2)/2)^2)
@assert rs_df.distances[4] ≈ sqrt(1^2+1^2)
@assert rs_df.distances[5] ≈ sqrt(3^2+1^2)/2

In [ ]:
Crystal.plot_distance_hist(rs, "fcc", dist_cutoff)

### bcc

The same as above but for bcc

In [ ]:
element = "Fe"
a = 1 #bcc_lattice_constants[element]
atoms, coords, box, box_size, box_vectors = Crystal.make_bcc_unitcell(element, a=a)
sc_atoms, sc_coords, sc_box, sc_box_size = Crystal.make_supercell(atoms, coords, box, box_size, nx=3, ny=3,nz=3)
n_atoms = length(sc_atoms);

In [ ]:
dist_cutoff = 2
rs_df, rs = Crystal.get_distance_df(sc_atoms, sc_box_size[1,1], sc_coords, dist_cutoff=dist_cutoff)

In [ ]:
@assert rs_df.distances[1] ≈ sqrt((sqrt(2)/2)^2 + 1/2^2)
@assert rs_df.distances[2] ≈ 1
@assert rs_df.distances[3] ≈ sqrt(2)
@assert rs_df.distances[4] ≈ sqrt((sqrt(2)/2)^2 + (3/2)^2)
@assert rs_df.distances[5] ≈ sqrt(sqrt(2)^2 + 1^2)

In [ ]:
Crystal.plot_distance_hist(rs, "bcc", dist_cutoff)